# Code de création d’images artificielles de météores

Ce code suit la feuille de route fournie par Jérémie et ses différentes étapes

## 1. Physique des météores

Cette partie à pour objectif de déterminer l’énergie lumineuse émise par un météoroïde lorsque celui-ci s'ablate en entrant dans l'atmosphère. 

On y lira les données simulées par le fichier "STEPS_int.dat0" fournies dans le doc. 

In [ ]:
#On importe les modules nécéssaires à la suite du code
import numpy as np 

#Lecture des fichiers de données simulées


#Code pour l'utilisation sur un ordinateur

file = np.loadtxt("STEPS_int.dat0").T #Lecture du fichier
dm_dt = file[11]                      #Extraction des données dm/dt (colonne 12)


"""
#Code pour l'utilisation sur le google colaboratory
dm_dt = np.array([ -0.23146095,-0.33088337,-0.47748998,-0.69555918,-1.0224216,
                  -1.5151257,-2.2591985,-3.3768978,-5.0244093,-7.3686303,
                  -10.317992,-12.974366,-12.461908,-4.7041475])
"""

#Valeur par défaut du pourcentage d'énergie totale convertie en lumière
tau = 0.05 
vitesse = 66000 #m/s 

#Calcul de l’énergie lumineuse émise, sous forme de tableau numpy
I = - tau * vitesse*vitesse * dm_dt
#Source : TP 

#Affichage des résultats
print(I)

#(ajouter les unités)

FileNotFoundError: ignored

## 2. Signal détecté

Cette partie vise à déterminer le signal détecté par la caméra à partir de différents paramètres que nous obtiendrons via des sources bibliographiques. 


Il faut corriger : 
- La distance entre le phénomène et le détecteur
- L’efficacité quantique du détecteur : Q=70%
- l’étalage de la lumière sur le détecteurs, via la PSF (point spread function), la taille des pixels et la longueur focale de la caméra.

Approximations :     
- On considère le météoride et la caméra alignés avec le centre de la Terre -> le météoride est pile au centre du champs de vision de la caméra

In [ ]:
#Paramètres déterminées via la bibliographie : 

h = 6.63*1e-34 #J/s
c = 3.00 * 1e8 #m/s
lambda_moyen = 600 * 1e-9 # m

altitude_meteorix = 500e3 # m 
#Source : Article meteorix
altitude_meteore = file[5] * 1e3 #m
distance_meteore_camera = np.sqrt((altitude_meteorix - altitude_meteore)**2 + (file[8])**2)

efficacite_quantique = 0.7 
#Fournie par le doc 

taille_pixels = 5*5 * 1e-12 #m²
#Source : Rapport 2022 (p12)

longueur_focale = 8 * 10e-3 #m 
#Source : Rapport 2022 (p11)

intensite_recue = I_emis / (distance_meteore_camera)**2
#Source : Discussion du 15/02

a = 0 
delta_temps = np.array([])
for i in file[1] : 
    delta_temps = np.append(delta_temps, i - a)
    a = i 
#boucle qui calcule l'écart de durée entre 2 lignes du fichier en secondes


nombre_photons = intensite_recue * taille_pixels * delta_temps * lambda_moyen /  (h * c)
#Nombre de pixels captés chaque intervalle de temps 

diametre_camera = 11.2 * 1e-3 #m 
#Source : Rapport de l'année dernière 

profondeur_camera = 5 * 1e-2 #m 
# Gros inconnu ! Difficile de lire le fichier de l'année dernière, je l'estime à l'ordre du centimètre en attendant

surface_airy = np.pi * (profondeur_camera * np.sin(1.22 * lambda_moyen / diametre_camera ) )**2 
#Calcul de la diffraction du a la camera en mètre

proportion_lumiere_par_pixel = taille_pixels / surface_airy #sans unité

signal_detecte = proportion_lumiere_par_pixel * efficacite_quantique * nombre_photons
# D'après jeremie, le signal detecté vaut S = f * Q  * np
# d'après analyse dimensionnelle : en W/m²

print(signal_detecte)

NameError: ignored

## 3. Image artificielle

Dans cette partie l'objectif est de créer des images de ce météoroïde en fonction du temps

In [1]:
from astropy.io import fits 
import matplotlib.pyplot as plt
%matplotlib inline
fig=plt.figure(figsize=(10,10))

images = np.resize(np.zeros(4300800),(14,480,640))
angle_camera = 40 #° 
largueur_image = np.abs(np.tan(angle_camera) * distance_meteore_camera) #m 

x_pixel = 640/2 + (640 * Xm / largueur_image )
y_pixel = (480 * Ym / largueur_image ) + 480/2
    
for i in range(len(largueur_image)) : 
    
    image_a_ajt = np.resize(np.zeros(307200),(480,640))

    image_a_ajt[int(y_pixel[i])][int(x_pixel[i])] = 255

    val_pixel_adj = (1 - proportion_lumiere_par_pixel) * 255/8 

    image_a_ajt[int(y_pixel[i]) +1][int(x_pixel[i])] = val_pixel_adj
    image_a_ajt[int(y_pixel[i])][int(x_pixel[i])+1 ] = val_pixel_adj
    image_a_ajt[int(y_pixel[i]-1)][int(x_pixel[i])] = val_pixel_adj
    image_a_ajt[int(y_pixel[i])][int(x_pixel[i])-1] = val_pixel_adj

    image_a_ajt[int(y_pixel[i]) +1][int(x_pixel[i]) +1] = val_pixel_adj
    image_a_ajt[int(y_pixel[0])-1][int(x_pixel[i])-1] = val_pixel_adj
    image_a_ajt[int(y_pixel[i])+1][int(x_pixel[i])-1] = val_pixel_adj
    image_a_ajt[int(y_pixel[i])-1][int(x_pixel[i])+1] = val_pixel_adj
    
    images[i] = image_a_ajt


import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(111)
fig.show()

ax.set_xlim(0,640)
ax.set_ylim(0,480)

for i in range(14):
    ax.imshow(images[i], cmap='gray')
    plt.pause(0.003)
    fig.canvas.draw()
plt.pause(.1)

NameError: ignored

<Figure size 720x720 with 0 Axes>